In [1]:
import pandas as pd

import glob2
import matplotlib.pyplot as plt
import datetime
import re
from datetime import datetime, timedelta
import itertools
%matplotlib inline

In [2]:
locale_name = "global"
read_enc_val ="ISO-8859-1"
read_enc_val ="cp1252"
read_enc_val ="utf-8"

In [3]:
zafilesog = pd.read_csv("Classification/all_files_ZAv1.csv")

In [4]:
skrfilesog = pd.read_csv("Classification/all_files_SKRv1.csv")

In [5]:
globalfilesog = pd.read_csv("Classification/all_files_"+locale_name+"v1.csv")

In [6]:
# zafilesog.columns

In [7]:
zafiles=zafilesog[['main_artist']]
skrfiles=skrfilesog[['main_artist']]
globalfiles=globalfilesog[['main_artist']]

zafiles = zafiles.drop_duplicates()
skrfiles = skrfiles.drop_duplicates()
globalfiles = globalfiles.drop_duplicates()

In [8]:
skrandza = zafiles.merge(skrfiles, left_on=["main_artist"], right_on=["main_artist"], how="inner")
skrandglobal = skrfiles.merge(globalfiles, left_on=["main_artist"], right_on=["main_artist"], how="inner")

mixedallthree = skrandza.merge(skrandglobal, left_on=["main_artist"], right_on=["main_artist"], how="inner")

In [9]:
zafiles.shape, skrfiles.shape, globalfiles.shape, skrandza.shape, skrandglobal.shape, mixedallthree.shape

((1156, 1), (697, 1), (1320, 1), (272, 1), (332, 1), (271, 1))

In [10]:
zacommon = zafilesog.merge(mixedallthree, left_on=["main_artist"], right_on=["main_artist"], how="inner")
skrcommon = skrfilesog.merge(mixedallthree, left_on=["main_artist"], right_on=["main_artist"], how="inner")
globalcommon = globalfilesog.merge(mixedallthree, left_on=["main_artist"], right_on=["main_artist"], how="inner")

In [11]:
zacommonboygroups = zacommon[zacommon["IsBoyGroup"]==True]["main_artist"].unique().tolist()
zacommongirlgroups = zacommon[zacommon["IsGirlGroup"]==True]["main_artist"].unique().tolist()

skrcommonboygroups = skrcommon[skrcommon["IsBoyGroup"]==True]["main_artist"].unique().tolist()
skrcommongirlgroups = skrcommon[skrcommon["IsGirlGroup"]==True]["main_artist"].unique().tolist()

globalcommonboygroups = globalcommon[globalcommon["IsBoyGroup"]==True]["main_artist"].unique().tolist()
globalcommongirlgroups = globalcommon[globalcommon["IsGirlGroup"]==True]["main_artist"].unique().tolist()

len(zacommonboygroups), len(zacommongirlgroups), len(skrcommonboygroups), len(skrcommongirlgroups), len(globalcommonboygroups), len(globalcommongirlgroups)

(9, 3, 9, 3, 9, 3)

In [12]:
zacommongirlgroups

['BLACKPINK', 'FIFTY FIFTY', 'The Ronettes']

In [13]:
zacommonboygroups

['5 Seconds of Summer',
 'AJR',
 'BTS',
 'Jonas Brothers',
 'Maroon 5',
 'OneRepublic',
 'Stray Kids',
 'The 1975',
 'V']

In [14]:
mixedallthree.to_csv("Classification/mixedallthreecommon.csv", index=False)

#### on the South African chart

In [15]:
zacommonthree = mixedallthree.merge(zafilesog, left_on=["main_artist"], right_on=["main_artist"], how="inner")

In [16]:
zacommonthree.columns

Index(['main_artist', 'trackAppearanceCount', 'artistAppearanceCount', 'rank',
       'uri', 'artist_names', 'track_name', 'source', 'peak_rank',
       'previous_rank', 'weeks_on_chart', 'streams', 'End Date', 'Week',
       'Year', 'End Date Dt', 'Seconds since Epoch', 'End Date Aggregated',
       'ArtistCount', 'isTopTen', 'Artist and Track', 'rank difference',
       'Position over Time', 'IsGirlGroup', 'IsBoyGroup'],
      dtype='object')

In [17]:
aggregates = zacommonthree.groupby(["main_artist", 'track_name']).agg(['min', 'max', 'count', 'nunique'])

In [18]:
aggregates.to_csv("Classification/zaaggregatesmixedallthreecommon.csv")

In [19]:
def summarisematchesinthree(zacommonthree, country):

    allmainartists = zacommonthree["main_artist"].unique().tolist()

    all_artist_dicts = []

    for artist in allmainartists:
        artist_dict = {}
        chartname = country
        artist_dict["main_artist"] = artist
        artist_dict["chart_country"] = chartname

        # Number of songs
        numsongs = len(zacommonthree[zacommonthree["main_artist"]==artist]["track_name"].unique().tolist())
        artist_dict["main_artist"] = artist
        artist_dict["num_songs_artist"] = numsongs

        # peak rank
        peakrank = zacommonthree[zacommonthree["main_artist"]==artist]["peak_rank"].min()
        artist_dict["peak_rank"] = peakrank

        # make the date time a datetime
        earlistenddate = zacommonthree[zacommonthree["main_artist"]==artist]["End Date"].min()
        artist_dict["Earliest End Date"] = earlistenddate
        latestenddate = zacommonthree[zacommonthree["main_artist"]==artist]["End Date"].max()
        artist_dict["Max End Date"] = latestenddate

        # TOTAL STREAMS
        totalstreams = zacommonthree[zacommonthree["main_artist"]==artist]["streams"].sum()
        artist_dict["total_streams"] = totalstreams

        # NUMBER YEARS APPEARANCE
        numyearsappearance = len(zacommonthree[zacommonthree["main_artist"]==artist]["Year"].unique().tolist())
        artist_dict["#_years_appeared"] = numyearsappearance

        # IS BOY GROUP 
        isbg = zacommonthree[zacommonthree["main_artist"]==artist]["IsBoyGroup"].unique().tolist()[0]
        artist_dict["IsBoyGroup"] = isbg

        # IS GIRL GROUP 
        isbg = zacommonthree[zacommonthree["main_artist"]==artist]["IsGirlGroup"].unique().tolist()[0]
        artist_dict["IsGirlGroup"] = isbg

        # NUMBER OF ARTIST APPEARANCES
        artistdf = zacommonthree[zacommonthree["main_artist"]==artist]
        numberofartistappearances = artistdf.groupby(["track_name"])["artistAppearanceCount"].max().reset_index()["artistAppearanceCount"].max()
        artist_dict["total_artistAppearanceCount"] = numberofartistappearances

        # Number of collaborators
        mainartists = zacommonthree[zacommonthree["main_artist"]==artist]["artist_names"].unique().tolist()
        allcollaborators = []
        for countartists in mainartists:
            thelistsincommoncollabs = list(set(countartists.split(", ")) - set([artist]))
            allcollaborators.append(thelistsincommoncollabs)

        mergedcollabs = list(itertools.chain(*allcollaborators))
        nummergedcollabs = len(list(set(mergedcollabs)))
        artist_dict["all_collaborators"] = nummergedcollabs

        all_artist_dicts.append(artist_dict)
        
    return all_artist_dicts

In [20]:
all_artist_dicts = summarisematchesinthree(zacommonthree, "ZA")
za_all_artist_combined_df = pd.DataFrame(all_artist_dicts)

all_artist_dicts = summarisematchesinthree(globalcommon, "Global")
global_all_artist_combined_df = pd.DataFrame(all_artist_dicts)


all_artist_dicts = summarisematchesinthree(skrcommon, "South Korea")
skr_all_artist_combined_df = pd.DataFrame(all_artist_dicts)

In [21]:
za_all_artist_combined_df.to_csv("Classification/commonchartsaggregates/za_aggregatesmixedallthreecommon.csv")

In [22]:
global_all_artist_combined_df.to_csv("Classification/commonchartsaggregates/global_aggregatesmixedallthreecommon.csv")

In [23]:
skr_all_artist_combined_df.to_csv("Classification/commonchartsaggregates/skr_aggregatesmixedallthreecommon.csv")

In [24]:
print("done.")

done.
